# Projet 3 – Anticiper la consommation énergétique de bâtiments

Vous mettrez en œuvre un modèle de régression pour prédire les besoins énergétiques de bâtiments publics afin de contribuer à la transition énergétique d'une ville.

🔧 Objectifs :
- Nettoyage automatisé des données
- Préparation des variables (mise à l’échelle, encodage)
- Entraînement et évaluation d’un modèle supervisé


In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

FILE = "ENB2012_data"
DATA = pd.read_excel(f"../data/{FILE}.xlsx")
print(DATA.columns)
print(DATA.shape)
DATA.head()

Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'Y1', 'Y2'], dtype='object')
(768, 10)


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


|Variable Name|Role|Type|Description|Units|Missing Values|
|-|-|-|-|-|-|
|X1|Feature|Continuous|Relative Compactness||no|
|X2|Feature|Continuous|Surface Area||no|
|X3|Feature|Continuous|Wall Area||no|
|X4|Feature|Continuous|Roof Area||no|
|X5|Feature|Continuous|Overall Height||no|
|X6|Feature|Integer|Orientation||no|
|X7|Feature|Continuous|Glazing Area||no|
|X8|Feature|Integer|Glazing Area Distribution||no|
|Y1|Target|Continuous|Heating Load||no|
|Y2|Target|Continuous|Cooling Load||no|